# (Hartmann-Hahn)^Ntimes-XY16-M at fixed tau

In [7]:
from collections import OrderedDict
import datetime
import numpy as np
import time
import os

In [8]:
try: 
    pulsedmasterlogic
except NameError:
    manager.startModule('logic', 'pulsedmasterlogic')
try: 
    scannerlogic
except NameError:
    manager.startModule('logic', 'scannerlogic')
try: 
    optimizerlogic
except NameError:
    manager.startModule('logic', 'optimizerlogic')
try:
    sequencegeneratorlogic
except NameError:
    manager.startModule('logic', 'sequencegeneratorlogic')
try:
    pulsedmeasurement
except NameError:
    manager.startModule('gui', 'pulsedmeasurement')   
try:
    poimanagerlogic
except NameError:
    manager.startModule('logic', 'poimaganerlogic')    

In [9]:
def write_to_logfile(nametag, timestamp, name, **kwargs):
    """ Write parameters to custom logfile with name nametag """
    if type(timestamp) is not str:
        timestamp = str(timestamp)
    parameters = list(kwargs)
    if len(parameters) == 1 and type(kwargs[parameters[0]]) is OrderedDict:
        param_dict = kwargs[parameters[0]]
        parameters = list(param_dict)
        kwargs = param_dict
    log_dir = pulsedmeasurement._save_logic.get_path_for_module('CustomLogfiles')
    log_path = os.path.join(log_dir, nametag + '.txt')
    if not os.path.isfile(log_path):
        with open(log_path, 'w') as logfile:
            logfile.write('# timestamp\t')
            logfile.write('# Name\t')
            for param in parameters:
                logfile.write(param + '\t')
            logfile.write('\n#\n')
    with open(log_path, 'a') as logfile:
        logfile.write(timestamp + '\t')
        logfile.write(name + '\t')
        for param in parameters:
            logfile.write('{0:3.6e}\t'.format(kwargs[param]))
        logfile.write('\n')
    return

In [10]:
# static hardware parameters:
setup = OrderedDict()
setup['sampling_freq'] = pulsedmeasurementlogic.sample_rate
setup['fc_binwidth'] = pulsedmeasurementlogic.fast_counter_binwidth
setup['wait_length'] = 1.5e-6
setup['aom_delay'] = 500e-9
#setup['channel_config_name'] = pulsedmeasurement.current_channel_config_name
setup['seq_trig']=''
setup['gate']='d_ch2'
setup['NV_name']='NV1'
setup['poi'] = poimanagerlogic.active_poi.get_key()
setup['laser_length'] = 3.0e-6
setup["min_counts"] = 10000 # if after a refocus only 40kcounts are measured the refocus is repeated up to max 3 times
# Set amplitude in logic and pulser hardware:
amp_dict = OrderedDict()
amp_dict['a_ch1'] = 0.25
amp_dict['a_ch2'] = 0.0
AWG5014C.amplitude_dict = amp_dict
#pulsedmaster.pulse_generator_settings_changed(setup['sampling_freq'], setup['channel_config_name'], amp_dict, False)

In [32]:
# for single frequency XY16-N:
hh_xy16_dip = OrderedDict()
hh_xy16_dip['mw_amp'] = 0.25
hh_xy16_dip['spinlock_amp'] = 0.1127
hh_xy16_dip['pol_transfer_time'] = 65.0*1e-6
hh_xy16_dip['num_of_pol_cycles'] = 50000
hh_xy16_dip['mw_freq'] = 100e6
hh_xy16_dip['order'] = 4
hh_xy16_dip['num_of_readouts'] = 10000
hh_xy16_dip['n_points'] = 5
hh_xy16_dip['nmr_position'] = 277.0e-9
hh_xy16_dip['measurement_time'] = 120*60
hh_xy16_dip['refocus_interval'] = 3*60
hh_xy16_dip['alternating']=True
hh_xy16_dip['rabi_period']=198.1*1e-9

In [38]:
# generate new Hartmann-Hahn + XY16 sequence at fixed position
sequencegeneratorlogic.delete_sequence('HH_XY16_dip')    
sequencegeneratorlogic.generate_HH_XY16_dip('HH_XY16_dip', hh_xy16_dip['rabi_period'], hh_xy16_dip['spinlock_amp'], 100e+6,
                                            0.25, hh_xy16_dip['pol_transfer_time'], hh_xy16_dip['num_of_pol_cycles'], 
                                            hh_xy16_dip['nmr_position'], hh_xy16_dip['order'], hh_xy16_dip['num_of_readouts'],
                                            hh_xy16_dip['n_points'], 'a_ch1', 3.0e-6,
                                            2.0, 0.7e-6, 1.0e-6, '', 'd_ch2', hh_xy16_dip['alternating'])
# sample, upload and load waveform
if 'HH_XY16_dip' not in sequencegeneratorlogic.saved_pulse_sequences:
    sequencegeneratorlogic.log.error('Sequence generation of HH_XY16_dip in notebook timed out.')
pulsedmasterlogic.sample_sequence('HH_XY16_dip', True)

while pulsedmasterlogic.status_dict['sauplo_sequence_busy'] or pulsedmasterlogic.status_dict['loading_busy']:
        time.sleep(5)

import winsound
frequency = 300  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

In [34]:
setup['poi'] = poimanagerlogic.active_poi.get_key()

In [35]:
refocus_poi(setup['poi'])

126819.666667 2.5934672187256566e-08


True

In [36]:
do_hh_xy16_dip_refocus(setup['poi'])

126819.666667 2.6853512479486333e-08
126819.666667 9.828332727820856e-09
126819.666667 2.1296296050214041e-10
126819.666667 9.508049726830026e-09
126819.666667 7.839276862066846e-10
126819.666667 1.62927836247655e-09
126819.666667 2.0402002643795975e-09
126819.666667 5.504517304236834e-09


True

In [37]:
# save measurement
pulsedmasterlogic.save_measurement_data('s', 'HH-'+str(hh_xy16_dip['num_of_pol_cycles'])+'_'
                                                +'XY16-'+str(hh_xy16_dip['order'])+'_'+setup['NV_name'], True, False)

'G:\\\\data\\\\QudiData\\2018\\06\\20180608\\PulsedMeasurement'

In [11]:
def refocus_poi(poi=None, max_drift=0.5,min_counts=setup["min_counts"]):
    if poi==None:
        return False
    scannerlogic.stop_scanning()
    poimanagerlogic.go_to_poi(poi)
    pos_start = scannerlogic.get_position()
    setup['NV_name']=poimanagerlogic.poi_list[poi]._name
    poimanagerlogic.optimise_poi(poi)
    while optimizerlogic.module_state() == 'idle':
        time.sleep(0.2)
    while optimizerlogic.module_state() != 'idle':
        time.sleep(0.2)
    time.sleep(5)
    pos_end=scannerlogic.get_position()
    print(counterlogic.countdata.mean(),abs(pos_end[0]-pos_start[0]))
    if (abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counterlogic.countdata.mean()<min_counts):
        scannerlogic.set_position('',x=pos_start[0],y=pos_start[1],z=pos_start[2])
        poimanagerlogic.set_new_position(poi,pos_start)
        print(abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counterlogic.countdata.mean()<min_counts)
        return False
        write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= setup['NV_name'], 
                     x_pos=pos_start[0], y_pos=pos_start[1], z_pos=pos_start[2],sucess=0)
    write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= setup['NV_name'], 
                     x_pos=pos_end[0], y_pos=pos_end[1], z_pos=pos_end[2],sucess=1)
    return True

def do_hh_xy16_dip(save_tag='',load_tag=''):
    
    pulsedmasterlogic.do_fit('No Fit')
    num_arr = np.linspace(1, hh_xy16_dip['num_of_pol_cycles'], hh_xy16_dip['n_points'])
    # set parameters in analysis tab
    if hh_xy16_dip['alternating']:
        pulsedmasterlogic.measurement_sequence_settings_changed(num_arr, hh_xy16_dip['n_points']*2, 100e-6, [],
                                                                hh_xy16_dip['alternating'])
    else:
        pulsedmasterlogic.measurement_sequence_settings_changed(num_arr, hh_xy16_dip['n_points'], 100e-6, [], 
                                                                hh_xy16_dip['alternating'])
    pulsedmasterlogic.fast_counter_settings_changed(setup['fc_binwidth'], setup['laser_length']+1e-6, hh_xy16_dip['n_points']*2)
    pulsedmasterlogic.analysis_interval_changed(3)
    pulsedmeasurementlogic.measurement_tag = save_tag
    # perform measurement
    pulsedmasterlogic.start_measurement(load_tag)
    while not pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.5)
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < hh_xy16_dip['measurement_time']:
        if not pulsedmasterlogic.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(0.5)
    pulsedmasterlogic.stop_measurement(save_tag)
    while pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.5)
    if save_tag=='':
        pulsedmasterlogic.save_measurement_data('s', 'HH-'+str(hh_xy16_dip['num_of_pol_cycles'])+'_'
                                                +'XY16-'+str(hh_xy16_dip['order'])+'_'+setup['NV_name'],True, False)
    
    time.sleep(2)
    return user_terminated

def do_hh_xy16_dip_refocus(poi):
    end_measure=False
    #refocus_poi(poi)
    hh_xy16_dip_total_time = hh_xy16_dip['measurement_time']
    hh_xy16_dip_runtime = 0.0
    hh_xy16_dip['measurement_time'] = hh_xy16_dip['refocus_interval']
    end_measure = do_hh_xy16_dip('hh_xy16_dip_refocus','')
    hh_xy16_dip_runtime += hh_xy16_dip['refocus_interval']
    while hh_xy16_dip_total_time > hh_xy16_dip_runtime:
        for i in range(1):
            end_measure = not refocus_poi(poi)
            if not end_measure:
                break
        if end_measure:
            break
        end_measure = do_hh_xy16_dip('hh_xy16_dip_refocus', 'hh_xy16_dip_refocus')
        hh_xy16_dip_runtime += hh_xy16_dip['refocus_interval']
        if end_measure:
            break
    hh_xy16_dip['measurement_time'] = hh_xy16_dip_total_time
    write_to_logfile('hh_xy16_dip_log', str(datetime.datetime.now()),name= setup['NV_name'],runtime=hh_xy16_dip_runtime)
    time.sleep(2)
    return end_measure